# TP : Convertisseur JPEG 2000 sur image complète

**Niveau :** Première NSI - TP avancé

**Prérequis :** Avoir fait le TP d'introduction sur 2 et 4 pixels

**Objectifs :**
- Appliquer la transformation de Haar sur une image complète
- Compresser une vraie image avec un seuil
- Calculer le taux de compression réel
- Visualiser les résultats

## Introduction

Dans le TP précédent, nous avons appris le principe de la transformation de Haar sur 2 puis 4 pixels.

Maintenant, nous allons appliquer cette transformation sur une **vraie image** complète !

### Le processus

1. Charger une image en niveaux de gris
2. Diviser l'image en blocs 2×2
3. Appliquer la transformation de Haar sur chaque bloc
4. Compresser en mettant à 0 les petites valeurs
5. Calculer le taux de compression obtenu

## Importation des bibliothèques

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

## Étape 1 : Chargement de l'image

**Consigne :** Placez votre image (en niveaux de gris) dans le même dossier que ce notebook.

**Note :** Pour que la transformation de Haar fonctionne, l'image doit avoir des dimensions paires (divisibles par 2).

In [ ]:
# TODO: Remplacer 'image.png' par le nom de votre image
nom_image = 'image.png'

# Chargement de l'image
image = Image.open(nom_image).convert('L')  # 'L' = niveaux de gris

# Conversion en tableau numpy
image_array = np.array(image, dtype=float)

# Vérification des dimensions
hauteur, largeur = image_array.shape
print(f"Image chargée : {largeur}×{hauteur} pixels")
print(f"Taille : {image_array.size} pixels")

# Vérifier que les dimensions sont paires
if hauteur % 2 != 0 or largeur % 2 != 0:
    print("⚠️ ATTENTION : Les dimensions doivent être paires !")
    print("   On va recadrer l'image...")
    # Recadrer pour avoir des dimensions paires
    hauteur = hauteur - (hauteur % 2)
    largeur = largeur - (largeur % 2)
    image_array = image_array[:hauteur, :largeur]
    print(f"   Nouvelle taille : {largeur}×{hauteur} pixels")

# Affichage de l'image
plt.figure(figsize=(8, 8))
plt.imshow(image_array, cmap='gray', vmin=0, vmax=255)
plt.title(f'Image originale ({largeur}×{hauteur} pixels)', fontsize=14, fontweight='bold')
plt.axis('off')
plt.colorbar(label='Niveau de gris')
plt.show()

## Étape 2 : Fonction de transformation de Haar

Nous réutilisons le principe du TP précédent : on transforme chaque bloc 2×2 en 4 coefficients.

In [ ]:
def transformation_haar_bloc(a, b, c, d):
    """
    Applique la transformation de Haar sur un bloc 2×2.
    
    Entrée : 4 pixels a, b, c, d
    
    Sortie : 4 coefficients (rouge, bleu, vert, violet)
    """
    # TODO: Compléter avec les formules de la transformation de Haar
    rouge =   # Moyenne globale
    bleu =    # Différences verticales
    vert =    # Différences horizontales
    violet =  # Différences obliques
    
    return rouge, bleu, vert, violet

# Test de la fonction
rouge, bleu, vert, violet = transformation_haar_bloc(134, 224, 137, 162)
print("Test de la fonction :")
print(f"  Rouge = {rouge:.2f}")
print(f"  Bleu = {bleu:.2f}")
print(f"  Vert = {vert:.2f}")
print(f"  Violet = {violet:.2f}")

## Étape 3 : Application sur l'image complète

On divise l'image en blocs 2×2 et on applique la transformation sur chaque bloc.

In [ ]:
def transformation_haar_image(image):
    """
    Applique la transformation de Haar sur une image complète.
    
    L'image est divisée en blocs 2×2, et chaque bloc est transformé.
    Le résultat est une image avec 4 quadrants :
    - Haut-gauche : moyennes (rouge)
    - Haut-droit : différences verticales (bleu)
    - Bas-gauche : différences horizontales (vert)
    - Bas-droit : différences obliques (violet)
    """
    hauteur, largeur = image.shape
    nouvelle_hauteur = hauteur // 2
    nouvelle_largeur = largeur // 2
    
    # Initialisation des 4 quadrants
    rouge = np.zeros((nouvelle_hauteur, nouvelle_largeur))
    bleu = np.zeros((nouvelle_hauteur, nouvelle_largeur))
    vert = np.zeros((nouvelle_hauteur, nouvelle_largeur))
    violet = np.zeros((nouvelle_hauteur, nouvelle_largeur))
    
    # TODO: Parcourir l'image par blocs 2×2
    for i in range(0, hauteur, 2):
        for j in range(0, largeur, 2):
            # Extraire les 4 pixels du bloc
            a = image[i, j]
            b = image[i, j+1]
            c = image[i+1, j]
            d = image[i+1, j+1]
            
            # Appliquer la transformation
            r, bl, v, vi = transformation_haar_bloc(a, b, c, d)
            
            # Stocker dans les quadrants
            rouge[i//2, j//2] = r
            bleu[i//2, j//2] = bl
            vert[i//2, j//2] = v
            violet[i//2, j//2] = vi
    
    # Reconstruction de l'image transformée
    resultat = np.zeros_like(image)
    resultat[0:nouvelle_hauteur, 0:nouvelle_largeur] = rouge
    resultat[0:nouvelle_hauteur, nouvelle_largeur:largeur] = bleu
    resultat[nouvelle_hauteur:hauteur, 0:nouvelle_largeur] = vert
    resultat[nouvelle_hauteur:hauteur, nouvelle_largeur:largeur] = violet
    
    return resultat, rouge, bleu, vert, violet

print("Transformation de l'image en cours...")
image_transformee, rouge, bleu, vert, violet = transformation_haar_image(image_array)
print("✓ Transformation terminée !")

## Étape 4 : Visualisation de la transformation

In [ ]:
# Visualisation des résultats
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Image originale
axes[0, 0].imshow(image_array, cmap='gray', vmin=0, vmax=255)
axes[0, 0].set_title("Image originale", fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

# Quadrant rouge (moyennes)
axes[0, 1].imshow(rouge, cmap='gray', vmin=0, vmax=255)
axes[0, 1].set_title("ROUGE : Moyennes\n(structure de l'image)", fontsize=12)
axes[0, 1].axis('off')

# Image transformée complète
axes[0, 2].imshow(image_transformee, cmap='gray')
axes[0, 2].set_title("Image transformée\n(4 quadrants)", fontsize=12)
axes[0, 2].axis('off')

# Pour visualiser les différences, on centre autour de 127.5
axes[1, 0].imshow(bleu + 127.5, cmap='gray', vmin=0, vmax=255)
axes[1, 0].set_title("BLEU : Différences verticales", fontsize=12)
axes[1, 0].axis('off')

axes[1, 1].imshow(vert + 127.5, cmap='gray', vmin=0, vmax=255)
axes[1, 1].set_title("VERT : Différences horizontales", fontsize=12)
axes[1, 1].axis('off')

axes[1, 2].imshow(violet + 127.5, cmap='gray', vmin=0, vmax=255)
axes[1, 2].set_title("VIOLET : Différences obliques", fontsize=12)
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("🎨 Interprétation :")
print("  • Le quadrant ROUGE contient la structure générale de l'image")
print("  • Les quadrants BLEU, VERT, VIOLET contiennent les détails (contours)")

## Étape 5 : Compression avec seuil

Maintenant, compressons en mettant à 0 les petites valeurs !

In [ ]:
def compresser(image_transformee, seuil):
    """
    Compresse l'image en mettant à 0 les valeurs dont
    la valeur absolue est inférieure au seuil.
    """
    compresse = image_transformee.copy()
    
    # TODO: Mettre à 0 les valeurs dont |valeur| < seuil
    masque = np.abs(compresse) < seuil
    compresse[masque] = 0
    
    return compresse

# Test avec différents seuils
seuils = [10, 20, 30]

fig, axes = plt.subplots(1, len(seuils)+1, figsize=(16, 4))

axes[0].imshow(image_transformee, cmap='gray')
axes[0].set_title("Transformée\n(sans compression)", fontsize=11)
axes[0].axis('off')

for i, seuil in enumerate(seuils):
    compresse = compresser(image_transformee, seuil)
    
    # Calcul du taux de compression
    nb_zeros = np.sum(compresse == 0)
    taux = (nb_zeros / compresse.size) * 100
    
    axes[i+1].imshow(compresse, cmap='gray')
    axes[i+1].set_title(f"Seuil = {seuil}\n{taux:.1f}% de zéros", fontsize=11)
    axes[i+1].axis('off')

plt.tight_layout()
plt.show()

## Étape 6 : Analyse détaillée de la compression

In [ ]:
# Choisir un seuil
seuil = 20

# Compresser
image_compresse = compresser(image_transformee, seuil)

# Statistiques
nb_total = image_compresse.size
nb_zeros = np.sum(image_compresse == 0)
nb_non_zeros = nb_total - nb_zeros
taux_compression = (nb_zeros / nb_total) * 100

print(f"📊 Analyse de la compression (seuil = {seuil}) :")
print(f"")
print(f"Nombre total de coefficients : {nb_total:,}")
print(f"Nombre de zéros après compression : {nb_zeros:,}")
print(f"Nombre de valeurs à stocker : {nb_non_zeros:,}")
print(f"")
print(f"💾 Taux de compression : {taux_compression:.1f}%")
print(f"💾 Facteur de compression : {nb_total/nb_non_zeros:.2f}x")
print(f"")
print(f"Interprétation :")
print(f"  • {taux_compression:.1f}% des coefficients sont mis à 0")
print(f"  • On ne stocke que {100-taux_compression:.1f}% des valeurs")
print(f"  • L'image compressée prend {100-taux_compression:.1f}% de l'espace original")

## Étape 7 : Comparaison des différents seuils

In [ ]:
# Tester plusieurs seuils
seuils_test = range(0, 51, 5)
taux_zeros = []

for s in seuils_test:
    comp = compresser(image_transformee, s)
    taux = (np.sum(comp == 0) / comp.size) * 100
    taux_zeros.append(taux)

# Graphique
plt.figure(figsize=(10, 6))
plt.plot(seuils_test, taux_zeros, 'b-o', linewidth=2, markersize=6)
plt.xlabel("Seuil de compression", fontsize=12)
plt.ylabel("Pourcentage de coefficients mis à zéro (%)", fontsize=12)
plt.title("Taux de compression en fonction du seuil", fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.show()

print("📈 Observations :")
print(f"  • Seuil 0 : {taux_zeros[0]:.1f}% de zéros (pas de compression)")
print(f"  • Seuil 25 : {taux_zeros[5]:.1f}% de zéros")
print(f"  • Seuil 50 : {taux_zeros[10]:.1f}% de zéros")
print(f"")
print(f"Plus le seuil augmente, plus on compresse (mais plus on perd de détails).")

## Conclusion

### Ce que nous avons réalisé

1. **Chargé une vraie image** en niveaux de gris
2. **Appliqué la transformation de Haar** sur toute l'image
3. **Compressé** en mettant à 0 les petites valeurs
4. **Calculé le taux de compression** réel obtenu

### Résultats typiques

Avec un seuil de 20 :
- Environ **60-70% de coefficients** mis à zéro
- **Facteur de compression** de 2-3x
- Perte de qualité **imperceptible à l'œil**

### Pour aller plus loin

Le vrai JPEG 2000 fait encore mieux grâce à :
- **Transformations multiples** : on réapplique la transformation sur les moyennes
- **Ondelettes plus sophistiquées** : Daubechies au lieu de Haar
- **Codage entropique** : les valeurs non-nulles sont codées efficacement
- **Résultat** : facteur de compression de **6-10x** avec qualité excellente !

Vous venez de créer votre propre convertisseur JPEG 2000 simplifié ! 🎉

---

**Source :** D'après un article de Christiane Rousseau, Université de Montréal  
**Adapté pour 1ère NSI - TP avancé sur image complète**